# 투숙일 별 호텔 예약 건수 예측
2015년 7월 ~ 2017년 6월, 2년간의 호텔 예약 데이터를 사용하여 2017년 7월과 8월동안 각 호텔의 투숙일별 예약 건수를 예측

## 1. Data Loading
Preprocessing을 위한 데이터 로딩

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('./hotel_reservation_data_task.csv')


## 2. Feature analysis
Correlation matrix를 기반으로 주어진 feature간의 상관관계를 분석

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Correlation matrix
corr_mat = data.corr()
print(corr_mat)

## 3. Pre-processing
데이터 전처리 수행 (n/a values 및 날짜 처리)

In [ ]:
# Replace n/a values as 0
data['num_children'].fillna(0, inplace=True)
data['agent_id'].fillna(0, inplace=True)
data['company_id'].fillna(0, inplace=True)
data['country'].fillna(0, inplace=True)

# date_arrivel = year + month + day
data['date_arrival'] = pd.to_datetime(data['date_of_arrival_year'].astype(str) + '-' +
                                      data['date_of_arrival_month'].astype(str) + '-' +
                                      data['date_of_arrival_day_of_month'].astype(str))


시간의 변화에 따른 예약 건수

In [ ]:
# Group data (hotel, arrival date)
hotel_arrival = data.groupby(['hotel', 'date_arrival']).size().reset_index(name='reserve_num')
print(hotel_arrival.head())

## 4. Prediction
### Time-series 기법들이 아래와 같이 다수 존재함

- **LSTM (Long Short-Term Memory)**: Recurrent neural network (RNN) 기반의 기본적인 time-series modeling 기법.
- **LLM (Large Language Model)**: Pre-trained된 대규모 언어 모델로써, time-series data 처리에 용이하나 데이터 크기 등을 고려하여 아래 ARIMA 모델을 사용.
- **ARIMA**: 통계 기반 모델로써, 이전 값들을 기반으로 time-series preding을 수행.


### ARIMA 모델이 해당 task에 적합한 이유

1. **데이터 적합성**: : 해당 모델은 단변량 시계열 예측에 적합함.
2. **구현 및 분석에 용이**: ARIMA는 LSTM에 비해 단순하고 구현 및 결과 해석에 용이함.
3. **성능**: 복잡하지 않은 데이터 특성을 고려할 때 좋은 성능을 발휘할 것이라 생각함.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Save predicted results
save_results = pd.DataFrame()

# Find unique hotel lists
hotels = hotel_arrival['hotel'].unique()

# Iteration for every hotel
for h in hotels:
    selected_data = hotel_arrival[hotel_arrival['hotel'] == h].set_index('date_arrival')
    
    # Training ARIMA with p=q=d=1
    model = ARIMA(selected_data['reserve_num'], order=(1, 1, 1))
    model_fit = model.fit()
    
    # Prediction from July to August 2017
    pred = model_fit.predict(start='2017-07-01', end='2017-08-31', typ='levels').reset_index()
    pred.columns = ['rentaldate', 'forecast_num_reservations']
    pred['hotel'] = h
    save_results = pd.concat([save_results, pred])

# Save the pred results
save_results = save_results[['hotel', 'rentaldate', 'forecast_num_reservations']]
save_results.to_csv('./forecast_result_format.csv', index=False)

## 5. Conclusion
ARIMA 모델을 기반으로 2017년 7월부터 8월까지 각 호텔의 예약 건수를 예측한 뒤 csv 파일에 저장함함